In [5]:
import os
import shutil
import re
import pandas as pd
from pandas import DataFrame
from pathlib import Path
from icecream import ic
import logging
import time

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
now_times = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

In [6]:
dir_path= '/mnt/nas1/lanwei-125/FGF5/disulfide/disulfide_peptide_cluster/cluster_0/all_refinement/'
result_path = f'{dir_path}/refinement/'
protein_pdb = '/mnt/nas1/lanwei-125/FGF5/dock_prepare/FGF5.pdb'
protein_chain = "A"
peptide_chain = "E"

In [7]:
cluster_dir_names = [
    dir_name for dir_name in Path(dir_path).iterdir()
    if dir_name.is_dir() and dir_name.name.split("_")[0] == "cluster"
]

pdb_names = [
    file.stem for cluster_dir in cluster_dir_names
    for file in cluster_dir.iterdir() if file.is_file() and file.suffix == ".pdb"
    ]
[str(cluster_dir_name) for cluster_dir_name in cluster_dir_names]

['/mnt/nas1/lanwei-125/FGF5/disulfide/disulfide_peptide_cluster/cluster_0/all_refinement/cluster_0']

In [ ]:
protein_lines = []
protein_chain = set()
with open(protein_pdb) as f:
    protein_lines = [line for line in f if line.startswith("ATOM") or line.startswith("TER")]
    for line in protein_lines:
        if line.startswith('ATOM') :
            chains = line[21]
            protein_chain.add(chains)
protein_chain_str = ','.join(protein_chain)

peptide_chain = set()
peptide_chain_str = ''
for cluster_dir_name in cluster_dir_names:
    pdb_path = Path(dir_path) / cluster_dir_name
    for pdb_file in pdb_path.glob('*.pdb'):
        parts = pdb_file.stem.split("_")
        if len(parts) >= 2 and parts[1] == "ref":
            continue

        if pdb_file.suffix == '.pdb' and '_ref' not in pdb_file.stem:
            # Read peptide lines
            peptide_lines = []
            with open(pdb_file) as f:
                peptide_lines = [line for line in f if line.startswith("ATOM")]
                for line in peptide_lines:
                    if line.startswith('ATOM') :
                        chains = line[21]
                        peptide_chain.add(chains)
            peptide_chain_str = ','.join(peptide_chain)

            # Write to new pdb file
            os.makedirs(pdb_path / pdb_file.stem, exist_ok=True)
            new_pdb_path = pdb_path / pdb_file.stem / (pdb_file.stem + '_ref.pdb')
            if new_pdb_path.exists():
                logger.info(f'{now_times}: {new_pdb_path} already exists')
                continue
            with open(new_pdb_path, 'w') as f:
                f.write(''.join(protein_lines))
                if protein_lines[-1].startswith('END'):
                    #删除END语句
                    del protein_lines[-1]
                if not protein_lines[-1].startswith('TER'):
                    f.write("\nTER\n")
                f.write(''.join(peptide_lines))
                f.write("\nTER\n")
                f.write("END\n")

            logger.info(f'{now_times}: processing {pdb_file} to {new_pdb_path}')

In [68]:
for cluster_dir_name in cluster_dir_names:

    for file in (Path(dir_path) / cluster_dir_name).glob('*.pdb'):
        pdb_file = file.stem
       
        work_dir = Path(dir_path) / cluster_dir_name / pdb_file
        for file in work_dir.glob('*.pdb'):
            refine_parts = file.stem.split('_')
            if len(refine_parts) >= 2 and refine_parts[1] == 'ref':

    

In [1]:
np_nums = "24"
for cluster_dir_name in cluster_dir_names:

    for file in (Path(dir_path) / cluster_dir_name).glob('*.pdb'):
        pdb_file = file.stem
       
        work_dir = Path(dir_path) / cluster_dir_name / pdb_file
        os.chdir(work_dir)
        for file in work_dir.glob('*.pdb'):
            if ('-ref' in file.stem) and ('_0250' not in file.stem):    
                ref_pdb = file.stem
                logger.info(f'{now_times}: processing {file}')

                f=open('prepack.flags', 'w')
                f.write("-s "+ ref_pdb + ".pdb \n")
                f.write("-flexpep_prepack \n")
                f.write("-nstruct 1 \n")
                f.write("-ex1 \n")
                f.write("-ex2aro \n")
                f.write("-mute core.io.database \n")
                f.write("-mute core.util.prof \n")
                f.write("-mute protocols.jd2.JobDistributor \n")
                f.write("-scorefile pack.score.sc ")
                f.close()
                os.system("mpirun -np "+ np_nums + " FlexPepDocking.mpi.linuxgccrelease @prepack.flags 2>&1 ")

                f=open('refinement.flags', 'w')
                f.write("-s "+ ref_pdb + "_0001.pdb \n")
                #f.write("-native "+ peptide + ".pdb \n")
                f.write("-flexPepDocking:receptor_chain "+protein_chain_str+" \n")
                f.write("-flexPepDocking:peptide_chain "+peptide_chain_str+" \n")
                f.write("-pep_refine \n")
                f.write("-use_input_sc \n")
                f.write("-nstruct 250 \n")
                f.write("-scorefile refinement.score.sc \n")
                f.write("-ex1 \n")
                f.write("-ex2aro \n")
                f.write("-mute core.io.database \n")
                f.write("-mute core.util.prof \n")
                f.write("-mute protocols.jd2.JobDistributor ")
                f.close()
                os.system(" mpirun.openmpi -np "+ np_nums + " FlexPepDocking.mpi.linuxgccrelease @refinement.flags 2>&1  ")
            


NameError: name 'cluster_dir_names' is not defined

In [27]:
result_path = Path(result_path)
if not result_path.exists():
    result_path.mkdir()


for file in (Path(dir_path) / cluster_dir_name).glob('*.pdb'):
    pdb_file = file.stem
    work_dir = Path(dir_path) / cluster_dir_name / pdb_file  
    score_file = work_dir / 'refinement.score.sc'

    with open(score_file, 'r') as f:
        headers = None
        rows = []
        lines = f.readlines()
        for line in lines:
            if line.startswith('SEQUENCE:'):
                continue
            elif line.startswith('SCORE:'):
                if headers is None:
                    headers = line.split()[1:]
                else:
                    rows.append(line.split()[1:])
            else:
                rows.append(line.split())
        df = pd.DataFrame(rows, columns=headers)
        df['reweighted_sc'] = df['reweighted_sc'].astype(float)
        df = df.sort_values(by='reweighted_sc', ignore_index=True)
        min_row = df.loc[df['reweighted_sc'].idxmin()]

    candidate_path = work_dir / f"{min_row['description']}.pdb"
    shutil.copy(candidate_path, result_path)

    result_score_file = result_path / 'candidate.txt'
    with open(result_score_file, 'a') as f:
        f.write(f"{min_row['description']}\t{min_row['reweighted_sc']}\n")

In [31]:
from utils_peptide.pdb_fixer import fixed_pdb_file
for file_entry in Path(result_path).iterdir():
    if file_entry.suffix == '.pdb':
        processed_pdb_path = Path(result_path) / file_entry.name
        candidate_pdb_path = Path(result_path) / f"{file_entry.stem.split('-')[0]}.pdb"
        candidate_pdb = []

        with open(processed_pdb_path) as f:
            lines = f.readlines()
            for line in lines:
                if line.startswith('ATOM') or line.startswith('TER') or line.startswith('END'):
                    candidate_pdb.append(line)

        with open(candidate_pdb_path, 'w') as f:
            f.write(''.join(candidate_pdb))

        fixed_pdb_file(candidate_pdb_path,result_path)

In [70]:
from math import log10


candidate_txt = Path("/mnt/nas1/lanwei-125/FGF5/disulfide/disulfide_peptide_cluster/cluster_0/all_refinement/refined/candidate.txt")
with open(candidate_txt, 'r') as f:
    lines = f.readlines()
    c = []
for line in lines:
    a = line.split()[0].split("-")[0]
    b = line.split()[1]
    if log10(abs(float(b))) < 4:
        newline = a + " " + b 
        c.append(newline)

In [98]:
df1 = pd.DataFrame(c)
d = df1[0].str.split(' ', expand=True)
df1['seq'] = d[0]
df1['value'] = d[1]
df1 = df1.drop(columns=[0])
df1.sort_values(by=['value'], inplace=True,ascending=False)
df1 = df1.reset_index(drop=True)
df1 = df1.iloc[:16]
a = df1['seq'].tolist()
a

['CNNILCPT',
 'EKCLGAFCFRRD',
 'QCNVPSCLT',
 'GKCHAAFCFRRD',
 'CTDKNCPLG',
 'CNSINCPLG',
 'GSCFDKECFRRD',
 'PCIGNTCPLG',
 'SCANINCGL',
 'CYIQECKLG',
 'CNGLPCVE',
 'GSCFGASCPRSI',
 'CTWSNCPLG',
 'CYKPDCPLG',
 'CYATWCPLG',
 'GSCSGAFCPRSI']

In [101]:
v1 = Path('/mnt/nas1/lanwei-125/FGF5/disulfide/MD/v1/')
s = []
for flie in v1.glob('*.pdb'):
    s.append(flie.stem)
set(a) & set(s)

{'GSCFDKECFRRD'}

In [86]:
from math import log10


candidate_txt = Path("/mnt/nas1/lanwei-125/FGF5/disulfide/disulfide_peptide_cluster/cluster_0/all_refinement/refined/candidate.txt")
with open(candidate_txt, 'r') as f:
    lines = f.readlines()
    c = []
for line in lines:
    a = line.split()[0].split("-")[0]
    b = line.split()[1]
    if log10(abs(float(b))) < 4:
        newline = a + " " + b 
        c.append(newline)
df1 = pd.DataFrame(c)
d = df1[0].str.split(' ', expand=True)
df1['seq'] = d[0]
df1['value'] = d[1]
df1 = df1.drop(columns=[0])
df1.sort_values(by=['value'], inplace=True,ascending=False)
df1 = df1.reset_index(drop=True)
df1 = df1.iloc[:16]
for seq in df1['seq']:
    os.umask(0) 
    HPEP = candidate_txt.parent/(seq + ".pdb")
    hpep_md = "/mnt/nas1/lanwei-125/FGF5/disulfide/MD/v3/"
    os.makedirs(hpep_md, exist_ok=True)
    shutil.copy(HPEP, hpep_md)